In [1]:
from __future__ import absolute_import, division, print_function

import itertools
import os

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
 
import tensorflow.keras.backend as K

import glob, os

from tensorflow import keras
import cv2

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Version:  2.4.3
Eager mode:  True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


In [2]:
import neptunecontrib.monitoring.kerastuner as npt_utils
from keras_tuner import HyperParameters, Objective

# CIFAR DATASET

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
nb_classes = 10
IMG_HEIGHT = 32
IMG_WIDTH = 32
y_train = np.squeeze(y_train)
y_test = np.squeeze(y_test)

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
y_test = keras.utils.to_categorical(y_test,10)
y_train = keras.utils.to_categorical(y_train,10)


In [4]:
x_test.shape

(10000, 32, 32, 3, 1)

In [5]:
y_test.shape

(10000, 10)

# SqueezeNet Model 

In [6]:
#hyper paramter squeezenet

 

from tensorflow.keras import layers
from keras_tuner import RandomSearch, Hyperband,SklearnTuner,BayesianOptimization
from tensorflow import keras
import numpy as np



def get_axis():
    axis = -1 if K.image_data_format() == 'channels_last' else 1
    return axis

def create_fire_module(x, nb_squeeze_filter, name, use_bypass=False):
    """
    Creates a fire module
    
    Arguments:
        x                 : input
        nb_squeeze_filter : number of filters of squeeze. The filtersize of expand is 4 times of squeeze
        use_bypass        : if True then a bypass will be added
        name              : name of module e.g. fire123
    
    Returns:
        x                 : returns a fire module
    """
    
    nb_expand_filter = 4 * nb_squeeze_filter
    squeeze    = layers.Conv2D(nb_squeeze_filter,(1,1), activation='relu', padding='same', name='%s_squeeze'%name)(x)
    expand_1x1 = layers.Conv2D(nb_expand_filter, (1,1), activation='relu', padding='same', name='%s_expand_1x1'%name)(squeeze)
    expand_3x3 = layers.Conv2D(nb_expand_filter, (3,3), activation='relu', padding='same', name='%s_expand_3x3'%name)(squeeze)
    
    axis = get_axis()
    x_ret =  layers.Concatenate(axis=axis, name='%s_concatenate'%name)([expand_1x1, expand_3x3])
    
    if use_bypass:
        x_ret =  layers.Add(name='%s_concatenate_bypass'%name)([x_ret, x])
        
    return x_ret


def output(x, nb_classes):
    x = layers.Conv2D(nb_classes, (1,1), strides=(1,1), padding='valid', name='conv10')(x)
    x = layers.GlobalAveragePooling2D(name='avgpool10')(x)
    x = layers.Activation("softmax", name='softmax')(x)
    return x

def build_SqueezeNet_fixed(hp):
    """
    Creating a SqueezeNet of version 1.0
    
    Arguments:
        input_shape  : shape of the input images e.g. (224,224,3)
        nb_classes   : number of classes
        use_bypass   : if true, bypass connections will be created at fire module 3, 5, 7, and 9 (default: False)
        dropout_rate : defines the dropout rate that is accomplished after last fire module (default: None)
        compression  : reduce the number of feature-maps (default: 1.0)
        
    Returns:
        Model        : Keras model instance
    """
    use_bypass = hp.Boolean('use_bypass')
    compression = hp.Fixed('compression',1.0)
    
    input_img = tf.keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

     
    x = tf.keras.layers.Conv2D(int(96*compression), (7,7), activation='relu', strides=(2,2), padding='same', name='conv1')(input_img)

    x = tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool1')(x)
    
    x = create_fire_module(x, int(16*compression), name='fire2')
    x = create_fire_module(x, int(16*compression), name='fire3', use_bypass=use_bypass)
    x = create_fire_module(x, int(32*compression), name='fire4')
    
    x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool4')(x)
    
    x = create_fire_module(x, int(32*compression), name='fire5', use_bypass=use_bypass)
    x = create_fire_module(x, int(48*compression), name='fire6')
    x = create_fire_module(x, int(48*compression), name='fire7', use_bypass=use_bypass)
    x = create_fire_module(x, int(64*compression), name='fire8')
    
    x =  layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool8')(x)
    
    x = create_fire_module(x, int(64*compression), name='fire9', use_bypass=use_bypass)

    dropout_rate = hp.Choice('dropout_rate',values=[0.0,0.5,0.8])
    
    if dropout_rate:
        x = tf.keras.layers.Dropout(dropout_rate)(x)
        
    x =  output(x, nb_classes)
    model = keras.Model(inputs=input_img, outputs=x)
    optimizer = hp.Choice("optimizer", ["RMSprop" "sgd"])
    model.compile(
        optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model
 
    
def build_SqueezeNet_11_fixed(hp):
    """
    Creating a SqueezeNet of version 1.1
    
    2.4x less computation over SqueezeNet 1.0 implemented above.
    
    Arguments:
        input_shape  : shape of the input images e.g. (224,224,3)
        nb_classes   : number of classes
        dropout_rate : defines the dropout rate that is accomplished after last fire module (default: None)
        compression  : reduce the number of feature-maps
        
    Returns:
        Model        : Keras model instance
    """
    
    use_bypass = hp.Boolean('use_bypass')
    compression = hp.Fixed('compression',1.0)
    
    input_img =  layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    
  
    x = layers.Conv2D(int(64*compression), (3,3), activation='relu', strides=(2,2), padding='same', name='conv1')(input_img)

    x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool1')(x)
    
    x = create_fire_module(x, int(16*compression), name='fire2')
    x = create_fire_module(x, int(16*compression), name='fire3')
    
    x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool3')(x)
    
    x = create_fire_module(x, int(32*compression), name='fire4')
    x = create_fire_module(x, int(32*compression), name='fire5')
    
    x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool5')(x)
    
    x = create_fire_module(x, int(48*compression), name='fire6')
    x = create_fire_module(x, int(48*compression), name='fire7')
    x = create_fire_module(x, int(64*compression), name='fire8')
    x = create_fire_module(x, int(64*compression), name='fire9')

    dropout_rate = hp.Choice('dropout_rate',values=[0.0,0.5,0.8])
        
    if dropout_rate:
        x = layers.Dropout(dropout_rate)(x)
    
    # Creating last conv10
    x = output(x, nb_classes)

 
    model = keras.Model(inputs=input_img, outputs=x)
    optimizer = hp.Choice("optimizer",["RMSprop","sgd"]) #["adam", "sgd"])
    model.compile(
        optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model



In [7]:
def squeeze_excitation_layer(input_layer, out_dim, ratio, conv=False):
  squeeze =  layers.GlobalAveragePooling2D()(input_layer)

  excitation =  layers.Dense(units=out_dim / ratio, activation='relu')(squeeze)
  excitation =  layers.Dense(out_dim,activation='sigmoid')(excitation)
  excitation =  tf.reshape(excitation, [-1,1,1,out_dim])

  scale =  layers.multiply([input_layer, excitation])

  if conv:
    shortcut = tf.keras.layers.Conv2D(out_dim,kernel_size=1,strides=1,
                                      padding='same',kernel_initializer='he_normal')(input_layer)
    shortcut = tf.keras.layers.BatchNormalization()(shortcut)
  else:
    shortcut = input_layer
    
  out = scale # tf.keras.layers.add([shortcut, scale])
  return out

# SqueezeNet SE Auto Model

In [8]:
#SqueezeNetSE auto model
from keras_tuner import HyperModel


class SqueezeNetSEAutoModel(HyperModel):
    def __init__(self, classes):
        self.classes = classes

    def build(self, hp):
       
          
        compression_val = hp.Fixed('compression',1.0)
    
        
        
        input_img =  layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

        
        x = layers.Conv2D(int(96*compression_val), (3,3), activation='relu', strides=(2,2), padding='same', name='conv1')(input_img)

        x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool1')(x)
    
         

        j = 2
        filter_size = 16

        num_fire = hp.Int("fire_module", 1,2, default=2)
        use_bypass = [ hp.Boolean('use_bypass'+str(i)) for i in range(num_fire)]
        pooling = [ hp.Choice('pooling'+str(i), ["max", "avg"]) for i in range(num_fire) ]
        
        print(use_bypass)
        print(pooling)
        for i in range(num_fire):
         
            ratio = hp.Choice("squeeze_ratio"+str(j),[8,16,32], default=32)
            x_in = x
            x = create_fire_module(x, int(filter_size*compression_val), name='fire'+str(j), )
            
            se_insert = hp.Boolean("SE_add"+str(j))
            
            if se_insert:
                x = squeeze_excitation_layer(x, out_dim=x.shape[3], ratio=ratio)
                
                if hp.Boolean("SE_skip"+str(j)):
                    if x_in.shape[3] != x.shape[3] :
                        x_in =  layers.Conv2D(x.shape[3],kernel_size=1,strides=1,
                                      padding='same',kernel_initializer='he_normal')(x_in)
                        x_in = tf.keras.layers.BatchNormalization()(x_in)
                        
                    x = tf.keras.layers.add([x_in, x])
                                     
            
            ratio = hp.Choice("squeeze_ratio"+str(j),[8,16,32], default=32)
            x_in = x
            x = create_fire_module(x, int(filter_size*compression_val), name='fire'+str(j+1),use_bypass=use_bypass[i])

            se_insert = hp.Boolean("SE_add"+str(j+1))
            
            if se_insert:
                 
                x = squeeze_excitation_layer(x, out_dim=x.shape[3], ratio=ratio)
                
                if hp.Boolean("SE_skip"+str(j+1)):
                    if x_in.shape[3] != x.shape[3] :
                        x_in =  layers.Conv2D(x.shape[3],kernel_size=1,strides=1,
                                      padding='same',kernel_initializer='he_normal')(x_in)
                        x_in = tf.keras.layers.BatchNormalization()(x_in)
                        
                    x = tf.keras.layers.add([x_in, x])
                     
            
                
            #if hp.Choice("pooling", ["max", "avg"]) == "max":
            if pooling[i] == "max":

                x =  layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool'+str(j+1))(x)
            else:
                x =  layers.AveragePooling2D(pool_size=(3,3), strides=(2,2), name='avgpool'+str(j+1))(x)


            j = j+2
            filter_size = filter_size+16
            
        num_fire2 = hp.Int('num_fire_2',0,2, default=2)
        use_bypass2 = [ hp.Boolean('use_bypass_2'+str(i)) for i in range(num_fire2)]
         
        print(use_bypass2)
         
        for i in range(num_fire2): 
            
            ratio = hp.Choice("squeeze_ratio"+str(j),[8,16,32], default=32)
            
            x_in = x
            x = create_fire_module(x, int(filter_size*compression_val), name='fire'+str(j))
            
            se_insert = hp.Boolean("SE_add"+str(j))
            
            if se_insert:
                x = squeeze_excitation_layer(x, out_dim=x.shape[3], ratio=ratio)
                if hp.Boolean("SE_skip"+str(j)):
                    if x_in.shape[3] != x.shape[3] :
                        x_in =  layers.Conv2D(x.shape[3],kernel_size=1,strides=1,
                                          padding='same',kernel_initializer='he_normal')(x_in)
                        x_in = tf.keras.layers.BatchNormalization()(x_in)
                    x = tf.keras.layers.add([x_in, x])
                
            ratio = hp.Choice("squeeze_ratio"+str(j),[8,16,32], default=32)
            
            x_in = x
            x = create_fire_module(x, int(filter_size*compression_val), name='fire'+str(j+1),use_bypass=use_bypass2[i])
            
            se_insert = hp.Boolean("SE_add"+str(j+1))
            
            if se_insert:
                x = squeeze_excitation_layer(x, out_dim=x.shape[3], ratio=ratio)
                if hp.Boolean("SE_skip"+str(j+1)):
                    if x_in.shape[3] != x.shape[3] :
                        x_in =  layers.Conv2D(x.shape[3],kernel_size=1,strides=1,
                                          padding='same',kernel_initializer='he_normal')(x_in)
                        x_in = tf.keras.layers.BatchNormalization()(x_in)
                    x = tf.keras.layers.add([x_in, x])
                    
            filter_size = filter_size+16
            j = j+2

            #x = create_fire_module(x, int(filter_size*compression_val), name='fire8')
            #x = create_fire_module(x, int(filter_size*compression_val), name='fire9',use_bypass=hp.Boolean('use_bypass'))
        
        
        
        dropout_rate = hp.Float('dropout_rate',0.0,0.8)
        if dropout_rate:
            x = layers.Dropout(dropout_rate)(x)

        x = output(x, self.classes)

        model = keras.Model(inputs=input_img, outputs=x)

        optimizer = hp.Choice("optimizer", ["RMSprop", "sgd"])
        model.compile(
            optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
        )
        return model
    

# SqueezeNet Model (Class definition)

In [9]:
#SqueezeNet 
from keras_tuner import HyperModel


class SqueezeNetModel(HyperModel):
    def __init__(self, classes):
        self.classes = classes

    def build(self, hp):
       
        use_bypass = hp.Boolean('use_bypass')
        compression = hp.Fixed('compression',1.0)

        input_img = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

        x = tf.keras.layers.Conv2D(int(96*compression), (7,7), activation='relu', strides=(2,2), padding='same', name='conv1')(input_img)

        x = tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool1')(x)

        x = create_fire_module(x, int(16*compression), name='fire2')
        x = create_fire_module(x, int(16*compression), name='fire3', use_bypass=use_bypass)
        x = create_fire_module(x, int(32*compression), name='fire4')

        x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool4')(x)

        x = create_fire_module(x, int(32*compression), name='fire5', use_bypass=use_bypass)
        x = create_fire_module(x, int(48*compression), name='fire6')
        x = create_fire_module(x, int(48*compression), name='fire7', use_bypass=use_bypass)
        x = create_fire_module(x, int(64*compression), name='fire8')

        x =  layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool8')(x)

        x = create_fire_module(x, int(64*compression), name='fire9', use_bypass=use_bypass)

        dropout_rate = hp.Choice('dropout_rate',values=[0.0,0.5,0.8])

        if dropout_rate:
            x = tf.keras.layers.Dropout(dropout_rate)(x)

        x =  output(x, self.classes)
        model = keras.Model(inputs=input_img, outputs=x)
        optimizer = hp.Choice("optimizer", ["RMSprop", "sgd"]) # adam is not good
        model.compile(
            optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
        )
        return model
 



# SqueezeNet 11 Model

In [10]:
#SqueezeNet 11 model
from keras_tuner import HyperModel


class SqueezeNet11Model(HyperModel):
    def __init__(self, classes):
        self.classes = classes

    def build(self, hp):
       
        use_bypass = hp.Boolean('use_bypass')
        compression = hp.Fixed('compression',1.0)

        input_img =  layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))


        x = layers.Conv2D(int(64*compression), (3,3), activation='relu', strides=(2,2), padding='same', name='conv1')(input_img)

        x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool1')(x)

        x = create_fire_module(x, int(16*compression), name='fire2')
        x = create_fire_module(x, int(16*compression), name='fire3')

        x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool3')(x)

        x = create_fire_module(x, int(32*compression), name='fire4')
        x = create_fire_module(x, int(32*compression), name='fire5')

        x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool5')(x)

        x = create_fire_module(x, int(48*compression), name='fire6')
        x = create_fire_module(x, int(48*compression), name='fire7')
        x = create_fire_module(x, int(64*compression), name='fire8')
        x = create_fire_module(x, int(64*compression), name='fire9')

        dropout_rate = hp.Choice('dropout_rate',values=[0.1,0.5,0.8])

        if dropout_rate:
            x = layers.Dropout(dropout_rate)(x)

        # Creating last conv10
        x = output(x, self.classes)
        model = keras.Model(inputs=input_img, outputs=x)
        optimizer = hp.Choice("optimizer",     ["sgd","RMSprop"])
        model.compile(
            optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
        )
        return model




In [ ]:
#https://gist.github.com/jeremyjordan/ac0229abd4b2b7000aca1643e88e0f02

#learning rate callbacks

import matplotlib.pyplot as plt

import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback


class LRFinder(Callback):
    
    '''
    A simple callback for finding the optimal learning rate range for your model + dataset. 
    
    # Usage
        ```python
            lr_finder = LRFinder(min_lr=1e-5, 
                                 max_lr=1e-2, 
                                 steps_per_epoch=np.ceil(epoch_size/batch_size), 
                                 epochs=3)
            model.fit(X_train, Y_train, callbacks=[lr_finder])
            
            lr_finder.plot_loss()
        ```
    
    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        epochs: Number of epochs to run experiment. Usually between 2 and 4 epochs is sufficient. 
        
    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: https://arxiv.org/abs/1506.01186

    '''
    
    def __init__(self, min_lr=1e-5, max_lr=1e-3, steps_per_epoch=None, epochs=None):
        super().__init__()
        
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.total_iterations = steps_per_epoch * epochs
        self.iteration = 0
        self.history = {}
        
    def clr(self):
        '''Calculate the learning rate.'''
        x = self.iteration / self.total_iterations 
        return self.min_lr + (self.max_lr-self.min_lr) * x
        
    def on_train_begin(self, logs=None):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.min_lr)
        
    def on_batch_end(self, epoch, logs=None):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.iteration += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.iteration)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
            
        K.set_value(self.model.optimizer.lr, self.clr())
 
    def plot_lr(self):
        '''Helper function to quickly inspect the learning rate schedule.'''
        plt.plot(self.history['iterations'], self.history['lr'])
        plt.yscale('log')
        plt.xlabel('Iteration')
        plt.ylabel('Learning rate')
        plt.show()
        
    def plot_loss(self):
        '''Helper function to quickly observe the learning rate experiment results.'''
        plt.plot(self.history['lr'], self.history['loss'])
        plt.xscale('log')
        plt.xlabel('Learning rate')
        plt.ylabel('Loss')
        plt.show()

In [12]:
import neptune
neptune.init(project_qualified_name='yyy',api_token='xx')

Project(cchantra/keras-tuner)

# Search schemes

In [62]:
#SqueezeSEAuto random
hp = HyperParameters()

import neptune

ratio = 0.1
PROJECT = "squeezeSEAutoRandom"+str(ratio)
neptune.create_experiment(PROJECT)

 
# This will override the `learning_rate` parameter with your
# own selection of choices

hp.Choice("learning_rate", values=[ 1e-3, 1e-4])



#fn_name = globals()["build_SqueezeNet_11_fixed"](hp)
# SqueezeNet 

mymodel = SqueezeNetSEAutoModel  (classes=nb_classes)

 
tuner = RandomSearch(
    #build_SqueezeNet_11_fixed,
    #build_squeezenet_auto_model,
    #build_model,
    
    hypermodel=mymodel,
    hyperparameters = hp,
    objective=Objective("val_accuracy", direction="max"),
    max_trials=100,
    executions_per_trial=1,
    overwrite=True,
    directory="mytest_dir",
    project_name=PROJECT,
    #distribution_strategy=tf.distribute.MirroredStrategy(),
    logger = npt_utils.NeptuneLogger(),
     
)
tuner.search_space_summary()

ratio = 0.1
EPOCH = 200
batch_size = 16
""""lr_finder = LRFinder(min_lr=1e-5, 
                    max_lr=1e-3,
                    steps_per_epoch=np.ceil(EPOCH/batch_size),
                    epochs=3)"""
my_callbacks = [
    tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10),
    #lr_finder,
    #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

tuner.search(x_train[:int(ratio*len(x_train))], y_train[:int(ratio*len(y_train))], epochs=EPOCH, callbacks = my_callbacks,validation_data=(x_test[:int(ratio*len(x_test))], y_test[:int(ratio*len(y_test))]))

models = tuner.get_best_models() #(num_models=10)
tuner.results_summary()
npt_utils.log_tuner_info(tuner)


#best_model = tuner.get_best_models(num_models=1)[0]
#best_model.evaluate(x_test, y_test)


Trial 100 Complete [00h 02m 19s]
val_accuracy: 0.5120000243186951

Best val_accuracy So Far: 0.5720000267028809
Total elapsed time: 03h 16m 32s
INFO:tensorflow:Oracle triggered exit
[True]
['avg']
[False]
Results summary
Results in mytest_dir/squeezeSEAutoRandom0.1
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.0001
compression: 1.0
fire_module: 1
use_bypass0: True
use_bypass1: True
pooling0: avg
pooling1: avg
squeeze_ratio2: 8
SE_add2: True
SE_add3: True
squeeze_ratio4: 32
SE_add4: True
SE_add5: False
num_fire_2: 1
use_bypass_20: False
use_bypass_21: False
squeeze_ratio6: 32
SE_add6: True
SE_add7: False
squeeze_ratio8: 16
SE_add8: False
SE_add9: False
dropout_rate: 0.3880199711045407
optimizer: RMSprop
SE_skip2: False
SE_skip4: True
SE_skip5: True
SE_skip3: False
SE_skip6: False
SE_skip7: True
SE_skip8: False
SE_skip9: False
Score: 0.5720000267028809
Trial summary
Hyperparameters:
learning_rate: 0.0001
compression

Unexpected error in ping thread.
Traceback (most recent call last):
  File "/home/chantana/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/home/chantana/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 384, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/home/chantana/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse()
  File "/home/chantana/anaconda3/lib/python3.7/http/client.py", line 1321, in getresponse
    response.begin()
  File "/home/chantana/anaconda3/lib/python3.7/http/client.py", line 296, in begin
    version, status, reason = self._read_status()
  File "/home/chantana/anaconda3/lib/python3.7/http/client.py", line 257, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/home/chantana/anaconda3/lib/python3.7/socket.py", line

In [ ]:
#SqueezeSEAuto random
hp = HyperParameters()

import neptune

ratio = 0.1
PROJECT = "squeezeSEAutoBayesian"+str(ratio)
neptune.create_experiment(PROJECT)

 
# This will override the `learning_rate` parameter with your
# own selection of choices

hp.Choice("learning_rate", values=[ 1e-3, 1e-4])



#fn_name = globals()["build_SqueezeNet_11_fixed"](hp)
# SqueezeNet 

mymodel = SqueezeNetSEAutoModel  (classes=nb_classes)

 
tuner = BayesianOptimization(
    #build_SqueezeNet_11_fixed,
    #build_squeezenet_auto_model,
    #build_model,
    
    hypermodel=mymodel,
    hyperparameters = hp,
    objective="val_accuracy",
    max_trials=100,
    executions_per_trial=1,
    overwrite=True,
    directory="mytest_dir",
    project_name= PROJECT,
    #distribution_strategy=tf.distribute.MirroredStrategy(),
    num_initial_points=2,
    alpha=0.0001,
    beta=2.6,
    seed=None,
    tune_new_entries=True,
    allow_new_entries=True,
    logger = npt_utils.NeptuneLogger(),
     
)
tuner.search_space_summary()


EPOCH = 200
batch_size = 16
""""lr_finder = LRFinder(min_lr=1e-5, 
                    max_lr=1e-3,
                    steps_per_epoch=np.ceil(EPOCH/batch_size),
                    epochs=3)"""
my_callbacks = [
    tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10),
    #lr_finder,
    #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

tuner.search(x_train[:int(ratio*len(x_train))], y_train[:int(ratio*len(y_train))], epochs=EPOCH, callbacks = my_callbacks,validation_data=(x_test[:int(ratio*len(x_test))], y_test[:int(ratio*len(y_test))]))

models = tuner.get_best_models() #(num_models=10)
tuner.results_summary()
npt_utils.log_tuner_info(tuner)


In [14]:
#SqueezeAuto Hyper
hp = HyperParameters()


# This will override the `learning_rate` parameter with your
# own selection of choices
hp.Choice("learning_rate", values=[ 1e-3, 1e-4])


import neptune
ratio = 0.1
PROJECT = "squeezeSEAutoHyper"+str(ratio)
neptune.create_experiment(PROJECT)

 
#fn_name = globals()["build_SqueezeNet_11_fixed"](hp)
# SqueezeNet 

mymodel = SqueezeNetSEAutoModel(classes=nb_classes)

 
EPOCH = 200
batch_size = 16
tuner = Hyperband(
    #build_SqueezeNet_11_fixed,
    #build_squeezenet_auto_model,
    #build_model,
    
    hypermodel=mymodel,
    hyperparameters = hp,
    objective="val_accuracy",
    #max_trials=20,
    executions_per_trial=1,
    overwrite=True,
    directory="mytest_dir",
    project_name=PROJECT,
    #distribution_strategy=tf.distribute.MirroredStrategy(),
    max_epochs=EPOCH,
    logger = npt_utils.NeptuneLogger(),
     
)
tuner.search_space_summary()


""""lr_finder = LRFinder(min_lr=1e-5, 
                    max_lr=1e-3,
                    steps_per_epoch=np.ceil(EPOCH/batch_size),
                    epochs=3)"""
my_callbacks = [
    tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10),
    #lr_finder,
    #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

tuner.search(x_train[:int(ratio*len(x_train))], y_train[:int(ratio*len(y_train))], epochs=EPOCH, callbacks = my_callbacks,validation_data=(x_test[:int(ratio*len(x_test))], y_test[:int(ratio*len(y_test))]))

models = tuner.get_best_models() #(num_models=10)
tuner.results_summary()

https://app.neptune.ai/cchantra/keras-tuner/e/KER-59
[False, False]
['max', 'max']
[False, False]
Search space summary
Default search space size: 24
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001], 'ordered': True}
compression (Fixed)
{'conditions': [], 'value': 1.0}
fire_module (Int)
{'default': 2, 'conditions': [], 'min_value': 1, 'max_value': 2, 'step': 1, 'sampling': None}
use_bypass0 (Boolean)
{'default': False, 'conditions': []}
use_bypass1 (Boolean)
{'default': False, 'conditions': []}
pooling0 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
pooling1 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
squeeze_ratio2 (Choice)
{'default': 32, 'conditions': [], 'values': [8, 16, 32], 'ordered': True}
SE_add2 (Boolean)
{'default': False, 'conditions': []}
SE_add3 (Boolean)
{'default': False, 'conditions': []}
squeeze_ratio4 (Choice)
{'default': 32, 'conditions': [], 

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/conv1/Relu (defined at /home/chantana/anaconda3/lib/python3.7/site-packages/keras_tuner/engine/tuner.py:147) ]] [Op:__inference_train_function_10245]

Function call stack:
train_function


In [16]:
#SqueezeAuto Bayes
hp = HyperParameters()


# This will override the `learning_rate` parameter with your
# own selection of choices
hp.Choice("learning_rate", values=[ 1e-3, 1e-4])


import neptune


neptune.create_experiment("squeezeAutoBaysian")
#fn_name = globals()["build_SqueezeNet_11_fixed"](hp)
# SqueezeNet 

mymodel = SqueezeNetAutoModel(classes=nb_classes)

 
    
ratio = 0.2
EPOCH = 200
batch_size = 16
tuner = BayesianOptimization(
    #build_SqueezeNet_11_fixed,
    #build_squeezenet_auto_model,
    #build_model,
    
    hypermodel=mymodel,
    hyperparameters = hp,
    objective="val_accuracy",
    max_trials=100,
    executions_per_trial=1,
    overwrite=True,
    directory="mytest_dir",
    project_name="squeezeAutoBayesian",
    #distribution_strategy=tf.distribute.MirroredStrategy(),
    num_initial_points=2,
    alpha=0.0001,
    beta=2.6,
    seed=None,
    tune_new_entries=True,
    allow_new_entries=True,
    logger = npt_utils.NeptuneLogger(),
     
)
tuner.search_space_summary()


""""lr_finder = LRFinder(min_lr=1e-5, 
                    max_lr=1e-3,
                    steps_per_epoch=np.ceil(EPOCH/batch_size),
                    epochs=3)"""
my_callbacks = [
    tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10),
    #lr_finder,
    #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

tuner.search(x_train[:int(ratio*len(x_train))], y_train[:int(ratio*len(y_train))], epochs=EPOCH, callbacks = my_callbacks,validation_data=(x_test[:int(ratio*len(x_test))], y_test[:int(ratio*len(y_test))]))

models = tuner.get_best_models() #(num_models=10)
tuner.results_summary()

Trial 20 Complete [00h 03m 25s]
val_accuracy: 0.5385000109672546

Best val_accuracy So Far: 0.5565000176429749
Total elapsed time: 01h 07m 08s
INFO:tensorflow:Oracle triggered exit
[True]
['avg']
[False, False]
Results summary
Results in mytest_dir/squeezeAutoBayesian
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.0001
compression: 1.0
fire_module: 1
use_bypass0: True
use_bypass1: True
pooling0: avg
pooling1: avg
num_fire_2: 2
use_bypass_20: False
use_bypass_21: False
dropout_rate: 0.0
optimizer: sgd
Score: 0.5565000176429749
Trial summary
Hyperparameters:
learning_rate: 0.0001
compression: 1.0
fire_module: 1
use_bypass0: True
use_bypass1: True
pooling0: avg
pooling1: avg
num_fire_2: 2
use_bypass_20: False
use_bypass_21: False
dropout_rate: 0.0
optimizer: sgd
Score: 0.5550000071525574
Trial summary
Hyperparameters:
learning_rate: 0.0001
compression: 1.0
fire_module: 1
use_bypass0: True
use_bypass1: True
pooling0: a

In [15]:
#SqueezeNet 11 Random

from keras_tuner import HyperParameters
hp = HyperParameters()

import neptune


neptune.create_experiment("squeeze11Random")

# This will override the `learning_rate` parameter with your
# own selection of choices
hp.Choice("learning_rate", values=[ 1e-3, 1e-4])



#fn_name = globals()["build_SqueezeNet_11_fixed"](hp)
# SqueezeNet 

mymodel = SqueezeNet11Model(classes=nb_classes)


tuner = RandomSearch(
    #build_SqueezeNet_11_fixed,
    #build_squeezenet_auto_model,
    #build_model,
    
    hypermodel=mymodel,
    hyperparameters = hp,
    objective=Objective("val_accuracy", direction="max"),
    max_trials=100,
    executions_per_trial=1,
    overwrite=True,
    directory="mytest_dir",
    project_name="squeeze11Random",
    #distribution_strategy=tf.distribute.MirroredStrategy(),
    logger = npt_utils.NeptuneLogger(),
     
)
tuner.search_space_summary()

ratio = 0.2
EPOCH = 200
batch_size = 16
""""lr_finder = LRFinder(min_lr=1e-5, 
                    max_lr=1e-3,
                    steps_per_epoch=np.ceil(EPOCH/batch_size),
                    epochs=3)"""
my_callbacks = [
    tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10),
    #lr_finder,
    #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

tuner.search(x_train[:int(ratio*len(x_train))], y_train[:int(ratio*len(y_train))], epochs=EPOCH, callbacks = my_callbacks,validation_data=(x_test[:int(ratio*len(x_test))], y_test[:int(ratio*len(y_test))]))

models = tuner.get_best_models() #(num_models=10)
tuner.results_summary()
npt_utils.log_tuner_info(tuner)

Trial 17 Complete [00h 07m 19s]
val_accuracy: 0.5065000057220459

Best val_accuracy So Far: 0.5379999876022339
Total elapsed time: 01h 22m 10s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in mytest_dir/squeeze11Random
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: False
compression: 1.0
dropout_rate: 0.1
optimizer: sgd
Score: 0.5379999876022339
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: True
compression: 1.0
dropout_rate: 0.1
optimizer: sgd
Score: 0.5335000157356262
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: False
compression: 1.0
dropout_rate: 0.5
optimizer: sgd
Score: 0.5304999947547913
Trial summary
Hyperparameters:
learning_rate: 0.001
use_bypass: True
compression: 1.0
dropout_rate: 0.5
optimizer: sgd
Score: 0.5295000076293945
Trial summary
Hyperparameters:
learning_rate: 0.001
use_bypass: False
compression: 1.0
dropout_rate: 0.1
opti

NeptuneException: 
[95m
----ClientHttpError-----------------------------------------------------------------------
[0m
Neptune server returned status [91m400[0m.

Server response was:
[91m{"title":"Length of stream does not match given range","code":400,"errorType":"BAD_REQUEST"}[0m

Verify the correctness of your call or contact Neptune support.

[92mNeed help?[0m-> https://docs.neptune.ai/getting-started/getting-help


In [20]:
#Squeeze11 Hyper
hp = HyperParameters()


# This will override the `learning_rate` parameter with your
# own selection of choices
hp.Choice("learning_rate", values=[ 1e-3, 1e-4])


import neptune


ratio = 0.2
PROJECT = "squeeze11"+str(ratio)
neptune.create_experiment(PROJECT)
 
#fn_name = globals()["build_SqueezeNet_11_fixed"](hp)
# SqueezeNet 

mymodel = SqueezeNet11Model(classes=nb_classes)

ratio = 0.2
EPOCH = 200
batch_size = 16
tuner = Hyperband(
    #build_SqueezeNet_11_fixed,
    #build_squeezenet_auto_model,
    #build_model,
    
    hypermodel=mymodel,
    hyperparameters = hp,
    objective="val_accuracy",
    #max_trials=20,
    executions_per_trial=1,
    overwrite=True,
    directory="mytest_dir",
    project_name=PROJECT,
    #distribution_strategy=tf.distribute.MirroredStrategy(),
    max_epochs=EPOCH,
    logger = npt_utils.NeptuneLogger(),
     
)
tuner.search_space_summary()


""""lr_finder = LRFinder(min_lr=1e-5, 
                    max_lr=1e-3,
                    steps_per_epoch=np.ceil(EPOCH/batch_size),
                    epochs=3)"""
my_callbacks = [
    tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10),
    #lr_finder,
    #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

tuner.search(x_train[:int(ratio*len(x_train))], y_train[:int(ratio*len(y_train))], epochs=EPOCH, callbacks = my_callbacks,validation_data=(x_test[:int(ratio*len(x_test))], y_test[:int(ratio*len(y_test))]))

models = tuner.get_best_models() #(num_models=10)
tuner.results_summary()

Trial 24 Complete [00h 00m 21s]
val_accuracy: 0.18299999833106995

Best val_accuracy So Far: 0.265500009059906
Total elapsed time: 00h 08m 11s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in mytest_dir/squeeze110.2
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.001
use_bypass: False
compression: 1.0
dropout_rate: 0.1
optimizer: RMSprop
tuner/epochs: 3
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
Score: 0.265500009059906
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: True
compression: 1.0
dropout_rate: 0.1
optimizer: RMSprop
tuner/epochs: 3
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
Score: 0.26249998807907104
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: True
compression: 1.0
dropout_rate: 0.5
optimizer: RMSprop
tuner/epochs: 3
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
Score: 0.2515000104904175
Trial summary
Hyperparameters:
lea

In [19]:
#SqueezeAuto Bayes
hp = HyperParameters()


# This will override the `learning_rate` parameter with your
# own selection of choices
hp.Choice("learning_rate", values=[ 1e-3, 1e-4])


import neptune


neptune.create_experiment("squeeze11Baysian")
#fn_name = globals()["build_SqueezeNet_11_fixed"](hp)
# SqueezeNet 

mymodel = SqueezeNet11Model(classes=nb_classes)

 
    
ratio = 0.2
EPOCH = 200
batch_size = 16
tuner = BayesianOptimization(
    #build_SqueezeNet_11_fixed,
    #build_squeezenet_auto_model,
    #build_model,
    
    hypermodel=mymodel,
    hyperparameters = hp,
    objective="val_accuracy",
    max_trials=200,
    executions_per_trial=1,
    overwrite=True,
    directory="mytest_dir",
    project_name="squeeze11Bayesian",
    #distribution_strategy=tf.distribute.MirroredStrategy(),
    num_initial_points=2,
    alpha=0.0001,
    beta=2.6,
    seed=None,
    tune_new_entries=True,
    allow_new_entries=True,
    logger = npt_utils.NeptuneLogger(),
     
)
tuner.search_space_summary()


""""lr_finder = LRFinder(min_lr=1e-5, 
                    max_lr=1e-3,
                    steps_per_epoch=np.ceil(EPOCH/batch_size),
                    epochs=3)"""
my_callbacks = [
    tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10),
    #lr_finder,
    #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

tuner.search(x_train[:int(ratio*len(x_train))], y_train[:int(ratio*len(y_train))], epochs=EPOCH, callbacks = my_callbacks,validation_data=(x_test[:int(ratio*len(x_test))], y_test[:int(ratio*len(y_test))]))

models = tuner.get_best_models() #(num_models=10)
tuner.results_summary()

Trial 200 Complete [00h 04m 14s]
val_accuracy: 0.5170000195503235

Best val_accuracy So Far: 0.5559999942779541
Total elapsed time: 18h 12m 21s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in mytest_dir/squeeze11Bayesian
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: True
compression: 1.0
dropout_rate: 0.8
optimizer: sgd
Score: 0.5559999942779541
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: True
compression: 1.0
dropout_rate: 0.8
optimizer: sgd
Score: 0.550000011920929
Trial summary
Hyperparameters:
learning_rate: 0.001
use_bypass: True
compression: 1.0
dropout_rate: 0.8
optimizer: sgd
Score: 0.546999990940094
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: False
compression: 1.0
dropout_rate: 0.8
optimizer: sgd
Score: 0.5450000166893005
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: False
compression: 1.0
dropout_rate: 0.8
opt

In [16]:
if tf.config.list_physical_devices('GPU'):
  strategy = tf.distribute.MirroredStrategy()
else:  # Use the Default Strategy
  strategy = tf.distribute.get_strategy()


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [12]:
#SqueezeNet Random
 
from keras_tuner import HyperParameters
hp = HyperParameters()
import neptune


ratio = 0.2
PROJECT = "squeezeRandom"+str(ratio)
neptune.create_experiment(PROJECT)
# This will override the `learning_rate` parameter with your
# own selection of choices
hp.Choice("learning_rate", values=[ 1e-3, 1e-4])



#fn_name = globals()["build_SqueezeNet_11_fixed"](hp)
# SqueezeNet 

mymodel = SqueezeNetModel(classes=nb_classes)


tuner = RandomSearch(
    #build_SqueezeNet_11_fixed,
    #build_squeezenet_auto_model,
    #build_model,
    
    hypermodel=mymodel,
    hyperparameters = hp,
    objective=Objective("val_accuracy", direction="max"),
    max_trials=100,
    executions_per_trial=1,
    overwrite=True,
    directory="mytest_dir",
    project_name=PROJECT,
     
    #distribution_strategy=strategy,
    logger = npt_utils.NeptuneLogger(),
     
)
tuner.search_space_summary()

 
EPOCH = 200
batch_size = 16
""""lr_finder = LRFinder(min_lr=1e-5, 
                    max_lr=1e-3,
                    steps_per_epoch=np.ceil(EPOCH/batch_size),
                    epochs=3)"""
my_callbacks = [
    tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10),
    #lr_finder,
    #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

tuner.search(x_train[:int(ratio*len(x_train))], y_train[:int(ratio*len(y_train))], epochs=EPOCH, callbacks = my_callbacks,validation_data=(x_test[:int(ratio*len(x_test))], y_test[:int(ratio*len(y_test))]))

models = tuner.get_best_models() #(num_models=10)
tuner.results_summary()

npt_utils.log_tuner_info(tuner)

Trial 17 Complete [00h 01m 11s]
val_accuracy: 0.10849999636411667

Best val_accuracy So Far: 0.5090000033378601
Total elapsed time: 00h 48m 43s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in mytest_dir/squeezeRandom0.2
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: False
compression: 1.0
dropout_rate: 0.5
optimizer: sgd
Score: 0.5090000033378601
Trial summary
Hyperparameters:
learning_rate: 0.001
use_bypass: False
compression: 1.0
dropout_rate: 0.8
optimizer: sgd
Score: 0.5084999799728394
Trial summary
Hyperparameters:
learning_rate: 0.001
use_bypass: False
compression: 1.0
dropout_rate: 0.5
optimizer: sgd
Score: 0.5080000162124634
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: False
compression: 1.0
dropout_rate: 0.8
optimizer: sgd
Score: 0.5009999871253967
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: False
compression: 1.0
dropout_rate: 0.0


In [15]:
strategy

In [ ]:
#Squeeze11 Hyper
hp = HyperParameters()


# This will override the `learning_rate` parameter with your
# own selection of choices
hp.Choice("learning_rate", values=[ 1e-3, 1e-4])


import neptune


ratio = 0.2
PROJECT = "squeeze11"+str(ratio)
neptune.create_experiment(PROJECT)
 
#fn_name = globals()["build_SqueezeNet_11_fixed"](hp)
# SqueezeNet 

mymodel = SqueezeNetModel(classes=nb_classes)

ratio = 0.2
EPOCH = 200
batch_size = 16
tuner = Hyperband(
    #build_SqueezeNet_11_fixed,
    #build_squeezenet_auto_model,
    #build_model,
    
    hypermodel=mymodel,
    hyperparameters = hp,
    objective="val_accuracy",
    #max_trials=20,
    executions_per_trial=1,
    overwrite=True,
    directory="mytest_dir",
    project_name=PROJECT,
    #distribution_strategy=tf.distribute.MirroredStrategy(),
    max_epochs=EPOCH,
    logger = npt_utils.NeptuneLogger(),
     
)
tuner.search_space_summary()


""""lr_finder = LRFinder(min_lr=1e-5, 
                    max_lr=1e-3,
                    steps_per_epoch=np.ceil(EPOCH/batch_size),
                    epochs=3)"""
my_callbacks = [
    tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10),
    #lr_finder,
    #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

tuner.search(x_train[:int(ratio*len(x_train))], y_train[:int(ratio*len(y_train))], epochs=EPOCH, callbacks = my_callbacks,validation_data=(x_test[:int(ratio*len(x_test))], y_test[:int(ratio*len(y_test))]))

models = tuner.get_best_models() #(num_models=10)
tuner.results_summary()

In [18]:
#SqueezeNet Hyper
hp = HyperParameters()

import neptune


ratio = 0.2
PROJECT = "squeezeHyper"+str(ratio)
neptune.create_experiment(PROJECT)

neptune.create_experiment(PROJECT)

# This will override the `learning_rate` parameter with your
# own selection of choices
hp.Choice("learning_rate", values=[ 1e-3, 1e-4])



#fn_name = globals()["build_SqueezeNet_11_fixed"](hp)
# SqueezeNet 

mymodel = SqueezeNetModel(classes=nb_classes)

EPOCH = 200
batch_size = 16
tuner = Hyperband(
    #build_SqueezeNet_11_fixed,
    #build_squeezenet_auto_model,
    #build_model,
    
    hypermodel=mymodel,
    max_trials=100,
    hyperparameters = hp,
    objective=Objective("val_accuracy", direction="max"),
    #max_trials=100,
    executions_per_trial=1,
    overwrite=True,
    directory="mytest_dir",
    project_name=PROJECT,
    #distribution_strategy=tf.distribute.MirroredStrategy(),
    max_epochs=EPOCH,
    logger = npt_utils.NeptuneLogger(),
)



    
tuner.search_space_summary()


""""lr_finder = LRFinder(min_lr=1e-5, 
                    max_lr=1e-3,
                    steps_per_epoch=np.ceil(EPOCH/batch_size),
                    epochs=3)"""
my_callbacks = [
    tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10),
    #lr_finder,
    #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

tuner.search(x_train[:int(ratio*len(x_train))], y_train[:int(ratio*len(y_train))], epochs=EPOCH, callbacks = my_callbacks,validation_data=(x_test[:int(ratio*len(x_test))], y_test[:int(ratio*len(y_test))]))

models = tuner.get_best_models() #(num_models=10)
tuner.results_summary()
npt_utils.log_tuner_info(tuner)

https://app.neptune.ai/cchantra/keras-tuner/e/KER-39
https://app.neptune.ai/cchantra/keras-tuner/e/KER-40


TypeError: __init__() got an unexpected keyword argument 'max_trials'

In [13]:
#SqueezeAuto Bayes
hp = HyperParameters()



# This will override the `learning_rate` parameter with your
# own selection of choices
hp.Choice("learning_rate", values=[ 1e-3, 1e-4])


import neptune

ratio = 0.2
PROJECT = "squeezeBayesian"+str(ratio)
neptune.create_experiment(PROJECT)

 
#fn_name = globals()["build_SqueezeNet_11_fixed"](hp)
# SqueezeNet 

mymodel = SqueezeNetModel(classes=nb_classes)

 
    
EPOCH = 200
batch_size = 16
tuner = BayesianOptimization(
    #build_SqueezeNet_11_fixed,
    #build_squeezenet_auto_model,
    #build_model,
    
    hypermodel=mymodel,
    hyperparameters = hp,
    objective="val_accuracy",
    max_trials=100,
    executions_per_trial=1,
    overwrite=True,
    directory="mytest_dir",
    project_name=PROJECT,
    #distribution_strategy=tf.distribute.MirroredStrategy(),
    num_initial_points=2,
    alpha=0.0001,
    beta=2.6,
    seed=None,
    tune_new_entries=True,
    allow_new_entries=True,
    logger = npt_utils.NeptuneLogger(),
     
)
tuner.search_space_summary()


""""lr_finder = LRFinder(min_lr=1e-5, 
                    max_lr=1e-3,
                    steps_per_epoch=np.ceil(EPOCH/batch_size),
                    epochs=3)"""
my_callbacks = [
    tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10),
    #lr_finder,
    #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

tuner.search(x_train[:int(ratio*len(x_train))], y_train[:int(ratio*len(y_train))], epochs=EPOCH, callbacks = my_callbacks,validation_data=(x_test[:int(ratio*len(x_test))], y_test[:int(ratio*len(y_test))]))

models = tuner.get_best_models() #(num_models=10)
tuner.results_summary()

Trial 100 Complete [00h 02m 26s]
val_accuracy: 0.47699999809265137

Best val_accuracy So Far: 0.5199999809265137
Total elapsed time: 05h 48m 29s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in mytest_dir/squeezeBayesian0.2
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: False
compression: 1.0
dropout_rate: 0.0
optimizer: sgd
Score: 0.5199999809265137
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: False
compression: 1.0
dropout_rate: 0.0
optimizer: sgd
Score: 0.515999972820282
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: False
compression: 1.0
dropout_rate: 0.0
optimizer: sgd
Score: 0.515500009059906
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: False
compression: 1.0
dropout_rate: 0.0
optimizer: sgd
Score: 0.515500009059906
Trial summary
Hyperparameters:
learning_rate: 0.0001
use_bypass: False
compression: 1.0
dropout_rate: 0.